## ONLY if running on Colaboratory, run this cell first (once)

In [ ]:
!git clone https://github.com/pie3636/newsjam.git
!mv newsjam/* .

## Install missing modules if needed (only run once)

In [ ]:
!python -m pip install -r requirements.txt
!python -m spacy download fr_core_news_sm
!python -m spacy download en_core_web_sm
# Note: You'll have to restart the kernel/runtime after running this cell

## Imports (only run once)

In [2]:
# MLSUM Corpus & CNN/Daily Mail Corpus
from datasets import load_dataset

# Loading article data
import json

# Our packages
from eval.rouge_l import RougeLEval
from eval.bert_eval import BERT_Eval
from eval.time import TimeEval

from summ.lsa import LSASummarizer
from summ.bert_embed import BertEmbeddingsSummarizer

from tqdm import tqdm

dataset_fr = load_dataset('mlsum', 'fr')
dataset_en = load_dataset('cnn_dailymail', '3.0.0')

rouge_l = RougeLEval()
bert = BERT_Eval()
timer = TimeEval()
lsa_summ = LSASummarizer()
flaubert_summ = BertEmbeddingsSummarizer('flaubert/flaubert_large_cased')
camembert_summ = BertEmbeddingsSummarizer('camembert/camembert-large')

Reusing dataset mlsum (/Users/josephkeenan/.cache/huggingface/datasets/mlsum/fr/1.0.0/77f23eb185781f439927ac2569ab1da1083195d8b2dab2b2f6bbe52feb600688)


Reusing dataset cnn_dailymail (/Users/josephkeenan/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at flaubert/flaubert_large_cased were not used when initializing FlaubertModel: ['pred_layer.proj.weight', 'pred_layer.proj.bias']
- This IS expected if you are initializing FlaubertModel from the checkpoint of a model trained on another task or with another 

## Summarize a single article

In [3]:
# Pick an article and its reference summary
article_fr = dataset_fr['test']['text'][54]
ref_summ_fr = dataset_fr['test']['summary'][54]

article_en = dataset_en['test']['article'][43]
ref_summ_en = dataset_en['test']['highlights'][43]

# Computes the summary and evaluation
# timer.evaluate_one(article, BertEmbeddingsSummarizer, 'camembert/camembert-large')

## Summarize a series of French articles

In [4]:
texts = dataset_fr['test']['text'][:50]
ref_summs = dataset_fr['test']['summary'][:50]

# Here we pick 5 articles
# gen_summs = []
# for text in tqdm(texts[:5]):
#     gen_summs.append(flaubert_summ.get_summary(text))

# scores1, scores2 = rouge_l.evaluate_many(ref_summs, gen_summs, 5)
# results = rouge_l.get_results(scores1, scores2)

# for k, v in results.items():
#     print(k.ljust(25), round(v*100, 3), '%')

timer.evaluate_many(texts, LSASummarizer)

100%|███████████████████████████████████████████| 50/50 [01:28<00:00,  1.76s/it]


1.7803929775000007

## Summarize a series of English articles

In [6]:
texts = dataset_en['test']['article'][:50]
ref_summs = dataset_en['test']['highlights'][:50]

# Here we pick 50 articles
gen_summs = []
for text in tqdm(texts[:50]):
     gen_summs.append(lsa_summ.get_summary(text, lang='en'))

scores1, scores2 = rouge_l.evaluate_many(ref_summs, gen_summs, 5)
results = rouge_l.get_results(scores1, scores2)

for k, v in results.items():
     print(k.ljust(25), round(v*100, 3), '%')

 88%|█████████████████████████████████████▊     | 44/50 [01:22<00:11,  1.89s/it]


KeyboardInterrupt: 

#### Optional: Save generated summaries to file

In [ ]:
with open('generated.txt', 'w') as f:
    for summ1, summ2 in tqdm(gen_summs):
        f.write(summ1)
        f.write('\n\n')
        f.write(summ2)
        f.write('\n\n')

## Summarize a series of scraped articles

In [5]:
with open('data/actu_preliminary.json', 'r', encoding='utf-8') as jsonfile:
    data = json.load(jsonfile)

texts = [article['text'] for article in data]
ref_summs = [article['summary'] for article in data]

gen_summs = []
for text in tqdm(texts):
    gen_summs.append(lsa_summ.get_summary(text))

scores1, scores2 = rouge_l.evaluate_many(ref_summs, gen_summs)
results = rouge_l.get_results(scores1, scores2)

for k, v in results.items():
    print(k.ljust(25), round(v*100, 3), '%')

100%|███████████████████████████████████████████| 47/47 [00:00<00:00, 95.80it/s]

Long precision avg        49.358 %
Long recall avg           61.974 %
Long F1-score avg         54.291 %
Keyword precision avg     48.302 %
Keyword recall avg        61.659 %
Keyword F1-score avg      53.503 %


## Implementation of BERTScore

Splitting summaries

In [11]:
long_summs, short_summs, ref_summs, key_ref_summs =  bert.split_summs(gen_summs, ref_summs, gen_keys=True)

Computation of BERTScore

In [12]:
bert.bert_score(long_summs, short_summs, ref_summs, key_ref_summs)

calculating scores...
computing bert embedding.



computing greedy matching.



done in 18.72 seconds, 2.51 sentences/sec
calculating scores...
computing bert embedding.



computing greedy matching.



done in 12.90 seconds, 3.64 sentences/sec


{'Long precision avg': '0.2326',
 'Long recall avg': '0.3982',
 'Long F1-score avg': '0.3125',
 'Keyword precision avg': '0.2440',
 'Keyword recall avg': '0.3905',
 'Keyword F1-score avg': '0.3151'}

Optional matrix of a score

In [ ]:
bert.get_matrix(long_summs, ref_summs, 4)

Experimentation w/ BERTScore

In [32]:
test_gen_summ = []
test_gen_summ.append('B')
print(len(test_gen_summ[0]))

print()

test_ref_summ = []
test_ref_summ.append('汉字')
print(len(test_ref_summ[0]))

1

2


In [33]:
bert.bert_score(test_gen_summ, test_ref_summ)

calculating scores...
computing bert embedding.



computing greedy matching.



done in 0.13 seconds, 7.84 sentences/sec


{'Long precision avg': '0.3093',
 'Long recall avg': '0.2388',
 'Long F1-score avg': '0.2743'}